### Validaciones = pipelines de procesamiento (mapeo de magnitudes cualitativas, normalización (metricas de estadistica dscriptiva e histograma) y validación de tipos de datos

## Este notebook fue utilizado para depurar los dataframes obtenidos en kaggle, reemplazando valores nulos, eliminado datos innecesarios o duplicados y demas

### Importamos las librerias y definimos configuraciones iniciales

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
pd.options.display.float_format='{:.2f}'.format
dir_export = '../clean_dataframes/{}'

### Realizamos las validaciones y correciones a los dataframes

#### Coinmarketcap (2020)

##### Cargamos el dataframe Coinmarketcap

In [3]:
coinmarketcap_data = pd.read_csv('../raw_dataframes/coinmarketcap.csv')
coinmarketcap_data_copy = coinmarketcap_data.copy()
coinmarketcap_data_copy.head()

,Unnamed: 0,24h_volume_usd,available_supply,id,last_updated,market_cap_usd,max_supply,name,percent_change_1h,percent_change_24h,percent_change_7d,price_btc,price_usd,rank,symbol,total_supply
0,0,22081300000,16785225,bitcoin,1515230661,284909052105,21000000.00,Bitcoin,-0.42,5.76,26.04,1.00,16973.80,1,BTC,16785225
1,1,5221370000,38739144847,ripple,1515230641,119207709132,100000000000.00,Ripple,-0.26,-9.23,24.15,0.00,3.08,2,XRP,99993093880
2,2,5705690000,96803840,ethereum,1515230649,100115499075,NaN,Ethereum,0.29,-1.04,45.01,0.06,1034.21,3,ETH,96803840
3,3,1569900000,16896225,bitcoin-cash,1515230652,44424061657,21000000.00,Bitcoin Cash,0.03,7.99,2.81,0.16,2629.23,4,BCH,16896225
4,4,428305000,25927070538,cardano,1515230654,25916647856,45000000000.00,Cardano,0.39,-5.87,64.99,0.00,1.00,5,ADA,31112483745


In [4]:
coinmarketcap_data_copy.dtypes

Unnamed: 0              int64
24h_volume_usd          int64
available_supply        int64
id                     object
last_updated            int64
market_cap_usd          int64
max_supply            float64
name                   object
percent_change_1h     float64
percent_change_24h    float64
percent_change_7d     float64
price_btc             float64
price_usd             float64
rank                    int64
symbol                 object
total_supply            int64
dtype: object

##### Convertimos los tipos Object a String

In [5]:
coinmarketcap_data_copy[['name','symbol']] = coinmarketcap_data_copy[['name','symbol']].astype('string') 
coinmarketcap_data_copy.dtypes

Unnamed: 0              int64
24h_volume_usd          int64
available_supply        int64
id                     object
last_updated            int64
market_cap_usd          int64
max_supply            float64
name                   string
percent_change_1h     float64
percent_change_24h    float64
percent_change_7d     float64
price_btc             float64
price_usd             float64
rank                    int64
symbol                 string
total_supply            int64
dtype: object

##### Dentro de este dataframe tenemos datos que se repiten en el dataframe siguiente. Como el siguiente más actual eliminamos las columnas con datos viejos y datos duplicados. También eliminamos aquellas columnas que no aportan información significativa

In [6]:
coinmarketcap_data_copy.drop(labels=['id', 'Unnamed: 0','last_updated', 'total_supply', 'percent_change_1h', 'percent_change_24h', 'percent_change_7d'], axis=1, inplace=True)
coinmarketcap_data_copy.head()

,24h_volume_usd,available_supply,market_cap_usd,max_supply,name,price_btc,price_usd,rank,symbol
0,22081300000,16785225,284909052105,21000000.00,Bitcoin,1.00,16973.80,1,BTC
1,5221370000,38739144847,119207709132,100000000000.00,Ripple,0.00,3.08,2,XRP
2,5705690000,96803840,100115499075,NaN,Ethereum,0.06,1034.21,3,ETH
3,1569900000,16896225,44424061657,21000000.00,Bitcoin Cash,0.16,2629.23,4,BCH
4,428305000,25927070538,25916647856,45000000000.00,Cardano,0.00,1.00,5,ADA


##### Reemplazamos los valores NaN de max_supply por inf ya que refleja mucho más fiel al comportamiento con las reservas totales de algunas cryptos. Es decir, no existe un limite de emisión para algunas criptos. Por ende para emular esa dirección usamos inf. 

##### Verificamos si existen valores nulos en el dataframe

In [7]:
coinmarketcap_data_copy[coinmarketcap_data_copy['available_supply'] == np.nan].value_counts()

Series([], dtype: int64)

In [8]:
coinmarketcap_data_copy.isna().value_counts()

24h_volume_usd  available_supply  market_cap_usd  max_supply  name   price_btc  price_usd  rank   symbol
False           False             False           True        False  False      False      False  False     73
                                                  False       False  False      False      False  False     27
dtype: int64

#### Prices (2022)

##### Cargamos el dataframe cryptocurrencies_prices

In [9]:
crypto_prices_data = pd.read_csv('../raw_dataframes/cryptocurrencies_prices.csv')
crypto_prices_data_copy = crypto_prices_data.copy()
crypto_prices_data_copy.head()

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,...,USD_percent_change_1h,USD_percent_change_24h,USD_percent_change_7d,USD_percent_change_30d,USD_percent_change_60d,USD_percent_change_90d,USD_market_cap,USD_market_cap_dominance,USD_fully_diluted_market_cap,USD_last_updated
0,1,Bitcoin,BTC,bitcoin,9175,2013-04-27T18:30:00Z,NaN,21000000.00,18965512.00,18965512.00,...,0.58,0.21,-14.03,7.93,-26.39,-33.28,718251756378.67,42.20,795300800946.06,2022-02-22T14:00:00Z
1,1027,Ethereum,ETH,ethereum,5516,2015-08-06T18:30:00Z,NaN,NaN,119688339.06,119688339.06,...,0.36,-1.19,-16.42,7.37,-36.56,-38.25,311420609004.72,18.30,311420609004.72,2022-02-22T14:00:00Z
2,825,Tether,USDT,tether,28381,2015-02-24T18:30:00Z,NaN,NaN,79354706739.88,82064697052.53,...,-0.00,0.01,-0.01,0.03,0.04,0.05,79396476158.26,4.67,82107892910.80,2022-02-22T14:00:00Z
3,1839,BNB,BNB,bnb,710,2017-07-24T18:30:00Z,NaN,165116760.00,165116760.89,165116760.89,...,0.95,-0.13,-14.62,-0.01,-32.69,-36.71,60889508005.13,3.58,60889507676.93,2022-02-22T14:00:00Z
4,3408,USD Coin,USDC,usd-coin,3082,2018-10-07T18:30:00Z,NaN,NaN,52722711494.02,52722711494.02,...,0.11,0.04,0.09,0.05,0.04,-0.09,52739691887.95,3.10,52739691887.95,2022-02-22T14:00:00Z


##### Imprimos los indices de las columnas para ver todas las dimensiones del dataframe

In [10]:
crypto_prices_data_copy.columns

Index(['id', 'name', 'symbol', 'slug', 'num_market_pairs', 'date_added',
       'tags', 'max_supply', 'circulating_supply', 'total_supply', 'cmc_rank',
       'self_reported_circulating_supply', 'self_reported_market_cap',
       'last_updated', 'platform_id', 'platform_name', 'platform_symbol',
       'platform_slug', 'platform_token_address', 'USD_price',
       'USD_volume_24h', 'USD_volume_change_24h', 'USD_percent_change_1h',
       'USD_percent_change_24h', 'USD_percent_change_7d',
       'USD_percent_change_30d', 'USD_percent_change_60d',
       'USD_percent_change_90d', 'USD_market_cap', 'USD_market_cap_dominance',
       'USD_fully_diluted_market_cap', 'USD_last_updated'],
      dtype='object')

##### Borramos tags y slug ya que son una columnas que no aportan

In [11]:
crypto_prices_data_copy['tags'].value_counts() #UNA COLUMNA VACIA

Series([], Name: tags, dtype: int64)

In [12]:
crypto_prices_data_copy.drop(['tags', 'slug', 'date_added'], axis=1, inplace=True)
crypto_prices_data_copy.head()

,id,name,symbol,num_market_pairs,max_supply,circulating_supply,total_supply,cmc_rank,self_reported_circulating_supply,self_reported_market_cap,...,USD_percent_change_1h,USD_percent_change_24h,USD_percent_change_7d,USD_percent_change_30d,USD_percent_change_60d,USD_percent_change_90d,USD_market_cap,USD_market_cap_dominance,USD_fully_diluted_market_cap,USD_last_updated
0,1,Bitcoin,BTC,9175,21000000.00,18965512.00,18965512.00,1,NaN,NaN,...,0.58,0.21,-14.03,7.93,-26.39,-33.28,718251756378.67,42.20,795300800946.06,2022-02-22T14:00:00Z
1,1027,Ethereum,ETH,5516,NaN,119688339.06,119688339.06,2,NaN,NaN,...,0.36,-1.19,-16.42,7.37,-36.56,-38.25,311420609004.72,18.30,311420609004.72,2022-02-22T14:00:00Z
2,825,Tether,USDT,28381,NaN,79354706739.88,82064697052.53,3,NaN,NaN,...,-0.00,0.01,-0.01,0.03,0.04,0.05,79396476158.26,4.67,82107892910.80,2022-02-22T14:00:00Z
3,1839,BNB,BNB,710,165116760.00,165116760.89,165116760.89,4,NaN,NaN,...,0.95,-0.13,-14.62,-0.01,-32.69,-36.71,60889508005.13,3.58,60889507676.93,2022-02-22T14:00:00Z
4,3408,USD Coin,USDC,3082,NaN,52722711494.02,52722711494.02,5,NaN,NaN,...,0.11,0.04,0.09,0.05,0.04,-0.09,52739691887.95,3.10,52739691887.95,2022-02-22T14:00:00Z


##### Reemplazamos los nan de max supply por inf debido a la razón comentada en el dataframe anterior

In [13]:
crypto_prices_data_copy['max_supply'].replace(np.nan, np.Inf, inplace=True)
crypto_prices_data_copy.head()

,id,name,symbol,num_market_pairs,max_supply,circulating_supply,total_supply,cmc_rank,self_reported_circulating_supply,self_reported_market_cap,...,USD_percent_change_1h,USD_percent_change_24h,USD_percent_change_7d,USD_percent_change_30d,USD_percent_change_60d,USD_percent_change_90d,USD_market_cap,USD_market_cap_dominance,USD_fully_diluted_market_cap,USD_last_updated
0,1,Bitcoin,BTC,9175,21000000.00,18965512.00,18965512.00,1,NaN,NaN,...,0.58,0.21,-14.03,7.93,-26.39,-33.28,718251756378.67,42.20,795300800946.06,2022-02-22T14:00:00Z
1,1027,Ethereum,ETH,5516,inf,119688339.06,119688339.06,2,NaN,NaN,...,0.36,-1.19,-16.42,7.37,-36.56,-38.25,311420609004.72,18.30,311420609004.72,2022-02-22T14:00:00Z
2,825,Tether,USDT,28381,inf,79354706739.88,82064697052.53,3,NaN,NaN,...,-0.00,0.01,-0.01,0.03,0.04,0.05,79396476158.26,4.67,82107892910.80,2022-02-22T14:00:00Z
3,1839,BNB,BNB,710,165116760.00,165116760.89,165116760.89,4,NaN,NaN,...,0.95,-0.13,-14.62,-0.01,-32.69,-36.71,60889508005.13,3.58,60889507676.93,2022-02-22T14:00:00Z
4,3408,USD Coin,USDC,3082,inf,52722711494.02,52722711494.02,5,NaN,NaN,...,0.11,0.04,0.09,0.05,0.04,-0.09,52739691887.95,3.10,52739691887.95,2022-02-22T14:00:00Z


##### Reemplazamos todos los valores restantes nan por la mediana, ya que es un estadistico no sesgado por valores extremos. Siendo el mercado un ecosistema volatil consideramos utilizarla para cualquier medida.

In [14]:
crypto_prices_data_copy.replace(np.nan, crypto_prices_data_copy.median, inplace=True)
crypto_prices_data_copy.isnull().value_counts()

id     name   symbol  num_market_pairs  max_supply  circulating_supply  total_supply  cmc_rank  self_reported_circulating_supply  self_reported_market_cap  last_updated  platform_id  platform_name  platform_symbol  platform_slug  platform_token_address  USD_price  USD_volume_24h  USD_volume_change_24h  USD_percent_change_1h  USD_percent_change_24h  USD_percent_change_7d  USD_percent_change_30d  USD_percent_change_60d  USD_percent_change_90d  USD_market_cap  USD_market_cap_dominance  USD_fully_diluted_market_cap  USD_last_updated
False  False  False   False             False       False               False         False     False                             False                     False         False        False          False            False          False                   False      False           False                  False                  False                   False                  False                   False                   False                   False           Fal

### Información básica

In [15]:
crypto_prices_data_copy.describe()

C:\Users\Augusto\anaconda3\envs\crypto-data-env\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


,id,num_market_pairs,max_supply,circulating_supply,total_supply,cmc_rank,USD_price,USD_volume_24h,USD_volume_change_24h,USD_percent_change_1h,USD_percent_change_24h,USD_percent_change_7d,USD_percent_change_30d,USD_percent_change_60d,USD_percent_change_90d,USD_market_cap,USD_market_cap_dominance,USD_fully_diluted_market_cap
count,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00
mean,6079.45,96.20,inf,1084328424218469.88,1144077404693127.50,500.50,423.89,177672818.20,27.32,0.32,-3.39,-14.31,-2.03,59.46,37.81,1783902440.81,0.10,1006276802311.09
std,3510.12,983.35,NaN,31119617733109572.00,31236621604292172.00,288.82,4137.15,2344450410.18,175.34,2.09,4.97,66.44,47.54,2442.23,1472.47,25093026693.46,1.47,31681580969534.14
min,1.00,1.00,0.00,302.00,0.00,1.00,0.00,0.00,-100.00,-27.71,-31.22,-64.63,-74.03,-92.81,-96.84,8513606.48,0.00,0.00
25%,3138.50,8.00,237209063.00,36056174.40,96502652.00,250.75,0.06,399671.52,-16.80,-0.03,-5.81,-23.16,-17.75,-51.85,-62.76,21202459.86,0.00,46947992.09
50%,5623.50,18.00,4455000000.00,188450892.01,500000000.00,500.50,0.39,2482520.20,1.12,0.28,-3.17,-17.83,-5.30,-40.68,-51.02,53723934.79,0.00,153923108.98
75%,8410.00,42.00,NaN,1067364352.81,2211890726.64,750.25,2.38,17482401.08,21.73,0.70,-0.96,-11.63,3.84,-26.04,-31.42,183048209.94,0.01,558454119.00
max,18112.00,28381.00,inf,981847688145643520.00,981847688145643520.00,1000.00,79515.81,65657024440.13,2883.11,28.84,45.97,2040.83,1007.91,75920.57,34464.88,718251756378.67,42.20,1001863052604233.62


In [16]:
crypto_prices_data_copy.dtypes

id                                    int64
name                                 object
symbol                               object
num_market_pairs                      int64
max_supply                          float64
circulating_supply                  float64
total_supply                        float64
cmc_rank                              int64
self_reported_circulating_supply     object
self_reported_market_cap             object
last_updated                         object
platform_id                          object
platform_name                        object
platform_symbol                      object
platform_slug                        object
platform_token_address               object
USD_price                           float64
USD_volume_24h                      float64
USD_volume_change_24h               float64
USD_percent_change_1h               float64
USD_percent_change_24h              float64
USD_percent_change_7d               float64
USD_percent_change_30d          

In [17]:
coinmarketcap_data_copy.describe()

,24h_volume_usd,available_supply,market_cap_usd,max_supply,price_btc,price_usd,rank
count,100.00,100.00,100.00,27.00,100.00,100.00,100.00
mean,531306623.36,43975344000.21,7608242773.06,311113981511.70,0.02,269.86,50.50
std,2354402554.51,263690597862.24,32344547236.05,1537153391033.22,0.10,1718.39,29.01
min,389519.00,645222.00,299515469.00,18900000.00,0.00,0.00,1.00
25%,19833325.00,54789875.50,461369198.00,68626123.00,0.00,0.72,25.75
50%,44427950.00,204527322.00,729732100.00,888000000.00,0.00,4.07,50.50
75%,171382750.00,1502972959.00,1980319073.00,10077785251.00,0.00,25.61,75.25
max,22081300000.00,2509983876433.00,284909052105.00,8000000000000.00,1.00,16973.80,100.00


#### Exportamos

In [18]:
coinmarketcap_data_copy.to_csv(dir_export.format('coinmarketcap_cleaned.csv'))
coinmarketcap_data_copy.to_csv(dir_export.format('cryptocurrencies_prices_cleaned.csv'))